In [2]:
import os
import subprocess
import pandas as pd #pip install pandas
import xlwt #pip install xlwt
from openpyxl import load_workbook #pip install openpyxldd
from tqdm import tqdm
import datetime

In [3]:
snmp_dir = "C:/Users/Никита Широкопетлев/Documents/SnmpGet/"
def get_IOD(wb, ser):
    IOD = {}
    work_sheet_IOD = wb['IOD']
    flag = False
    counter = 0
    for rows in work_sheet_IOD.iter_rows(values_only=True):
        if rows[2] == ser[:2] and flag == False:
            #print('Модель:', rows[1])
            flag = True
        elif flag == True and counter < 9:
            if rows[0] and rows[0] != 'Модель':
                IOD[rows[0]] = rows[1]
            counter += 1
    return IOD


#def get_info_row(wb, row, cell_ser = 3, cell_IP = 2):
#    return result

def search_cell(name_cell, work_sheet):
    for cell in range(1, 100):
        if work_sheet.cell(row = 1, column = cell).value == name_cell:
            break
    print('Столбец "', name_cell, '": ', cell, sep = '')
    return cell

In [4]:
wb = load_workbook('./printers.xlsx')
print(wb.sheetnames)
work_sheet = wb['MAIN']
cell_ser = search_cell('Серийный номер', work_sheet) - 1
cell_IP = search_cell('IP', work_sheet) - 1

['MAIN', 'IOD', 'AL75023437', 'CNB1L7J286', 'CNDRNBH78N', 'E75198L1N182781', 'E75373J7N556962', 'E75373J7N561816', 'E75373J7N561817', 'E75373J7N561836', 'VA87800338', 'VCF7538360', 'VCG8887941', 'VCG8X00618', 'VCG8Y01076', 'VCG9118770', 'ZDP1BJAF80005JJ', 'ZDP1BJFF9000RLT', 'ZDP1BJFF9000TJP', 'ZDP1BJFF9000VXY', 'ZDP1BJFF9000VYX', 'WVV11120']
Столбец "Серийный номер": 2
Столбец "IP": 3


In [5]:
start_row = 2
current_sheet = 3
for row in tqdm(work_sheet.iter_rows(min_row=2, values_only=True)):
    
    work_sheet_pr = wb[wb.sheetnames[current_sheet]]

    if row[0]:
        pass
    else:
        break
    result = []
    IOD = {} 
    try:
        IOD = get_IOD(wb, ser=row[cell_ser])
        #print(IOD)
        for key, value in IOD.items():
            res = subprocess.check_output("./SnmpGet.exe -r:" + str(row[cell_IP]) + 
                                      ' -o:' + value).decode("cp866").split('\n')
            result.append(res[-2].split('=')[-1])
            IOD[key] = str(res[-2].split('=')[-1]).strip('\r')
        #print(IOD)
        #work_sheet_pr = wb[IOD['Серийный номер принтера']]
        cur_row = 1
        flag = False
        for row_pr in work_sheet_pr.iter_rows(values_only=True):
            if row_pr[0] in IOD:
                try:
                    work_sheet_pr.cell(row=cur_row, column=2).value = int(IOD[row_pr[0]])
                except:
                    work_sheet_pr.cell(row=cur_row, column=2).value = IOD[row_pr[0]]
            
            if row_pr[0] == 'Status':
                work_sheet_pr.cell(row=cur_row, column=2).value = 'online'
            if flag:
                
                try:
                    tmp = str(row_pr[0].split()[0])
                except:
                    tmp = row_pr[0]
                    #print(type(tmp))
                
                if tmp == str(datetime.datetime.now().strftime("%d.%m.%Y")) or not tmp:
                    #print(IOD)
                    work_sheet_pr.cell(row=cur_row, column=1).value = str(datetime.datetime.now().strftime("%d.%m.%Y"))
                    work_sheet_pr.cell(row=cur_row, column=2).value = int(IOD['Оставшееся число копий тонера'])/int(IOD['Максимальное число копий тонера'])
                    work_sheet_pr.cell(row=cur_row, column=3).value = int(IOD['Кол-во напечатанных страниц'])
                    flag = False

            if row_pr[0] == 'Дата':
                flag = True

            cur_row += 1

            
    except:
        work_sheet_pr = wb[wb.sheetnames[current_sheet]]
        cur_row = 1
        for row_pr in work_sheet_pr.iter_rows(values_only=True):
            if row_pr[0] == 'Status':
                if IOD['Серийный номер принтера'][0] == '%':
                    work_sheet_pr.cell(row=cur_row, column=2).value = 'offline'
                    print(IOD)
                    print("Printer", row[1], 'offline')
                else:
                    work_sheet_pr.cell(row=cur_row, column=2).value = 'failed'
                    print(IOD)
                    print("Printer", row[1], 'failed')
            cur_row += 1 

    current_sheet += 1
    

1it [00:05,  5.97s/it]

{'Серийный номер принтера': '%Failed to get value of SNMP variable. Timeout.', 'Цвет тонера': '%Failed to get value of SNMP variable. Timeout.', 'Максимальное число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Оставшееся число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Кол-во напечатанных страниц': '%Failed to get value of SNMP variable. Timeout.', 'Название картриджа': '%Failed to get value of SNMP variable. Timeout.', 'Название принтера': '%Failed to get value of SNMP variable. Timeout.'}
Printer CNB1L7J286 offline


3it [01:21, 30.42s/it]

{'Серийный номер принтера': '%Failed to get value of SNMP variable. Timeout.', 'Цвет тонера': '%Failed to get value of SNMP variable. Timeout.', 'Максимальное число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Оставшееся число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Кол-во напечатанных страниц': '%Failed to get value of SNMP variable. Timeout.', 'Название картриджа': '%Failed to get value of SNMP variable. Timeout.', 'Название принтера': '%Failed to get value of SNMP variable. Timeout.'}
Printer CNDRNBH78N offline


5it [01:59, 25.27s/it]

...

{'Серийный номер принтера': '%Failed to get value of SNMP variable. Timeout.', 'Цвет тонера': '%Failed to get value of SNMP variable. Timeout.', 'Максимальное число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Оставшееся число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Кол-во напечатанных страниц': '%Failed to get value of SNMP variable. Timeout.', 'Название картриджа': '%Failed to get value of SNMP variable. Timeout.', 'Название принтера': '%Failed to get value of SNMP variable. Timeout.'}
Printer VA87800338 offline


15it [03:22, 13.72s/it]

{'Серийный номер принтера': '%Failed to get value of SNMP variable. Timeout.', 'Цвет тонера': '%Failed to get value of SNMP variable. Timeout.', 'Максимальное число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Оставшееся число копий тонера': '%Failed to get value of SNMP variable. Timeout.', 'Кол-во напечатанных страниц': '%Failed to get value of SNMP variable. Timeout.', 'Название картриджа': '%Failed to get value of SNMP variable. Timeout.', 'Название принтера': '%Failed to get value of SNMP variable. Timeout.'}
Printer WVV11120 offline


19it [03:28, 10.97s/it]


IndexError: list index out of range

In [6]:
wb.save('./printers.xlsx')